In [1]:
from IPython import display
import ultralytics
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
display.clear_output()
ultralytics.checks()

Ultralytics YOLOv8.2.99  Python-3.11.7 torch-2.2.0+cpu CPU (AMD Ryzen 7 5825U with Radeon Graphics)
Setup complete  (16 CPUs, 15.3 GB RAM, 48.0/238.0 GB disk)


In [13]:
model = YOLO("yolov8n-seg.yaml").load("yolov8n-seg.pt")

Transferred 417/417 items from pretrained weights


In [3]:
model = YOLO('./runs/segment/train7/weights/best.pt')

In [ ]:
# freeze 10 layers
freeze = 0

freeze = [f'model.{x}.' for x in range(freeze)] # สร้างลิสต์ของชื่อเลเยอร์ที่ต้องการล็อค
for k, v in model.named_parameters(): # วนลูปผ่านทุกพารามิเตอร์ในโมเดล
    
    # เช็คว่าชื่อของพารามิเตอร์ (k) มีคำว่า model.{x}. ใด ๆ ในลิสต์ freeze หรือไม่
    if any(x in k for x in freeze):
        print(f'freezing {k}')
        v.requires_grad = False # ปิดการอัปเดตน้ำหนักของเลเยอร์นั้น
    else:
        print(f'unfreeze {k}')
        v.requires_grad = True

In [7]:
import torch

optimizer = torch.optim.SGD(
    filter(lambda p: p.requires_grad, model.parameters()), # เลือกพารามิเตอร์ของโมเดลที่ไม่ได้ล็อค เพื่อ update ค่า weight
    lr=1e-4,
    momentum=0.9 
)

In [ ]:
# train model
result = model.train(
    batch=-1,
    device="cpu",
    data="./setting.yaml",
    epochs=100,
    imgsz=640,
    cache=True
)

In [ ]:
# train model ต่อจากที่หยุด
model = YOLO('./runs/segment/train7/weights/last.pt')
results = model.train(resume = True)

In [ ]:
# load model
model = YOLO('./runs/segment/train8/weights/best.pt')

# แปลงโมเดลเป็น ncnn
model.export(format="ncnn")

In [ ]:
# Use Raspberry Pi Camera

rpicam-hello # test camera

# Inference with Camera
import cv2
from picamera2 import Picamera2

from ultralytics import YOLO

# Initialize the Picamera2
picam2 = Picamera2()
picam2.preview_configuration.main.size = (1280, 720)
picam2.preview_configuration.main.format = "RGB888"
picam2.preview_configuration.align()
picam2.configure("preview")
picam2.start()

# Load the YOLO11 model
model = YOLO("yolo11n.pt")

while True:
    # Capture frame-by-frame
    frame = picam2.capture_array()

    # Run YOLO11 inference on the frame
    results = model(frame)

    # Visualize the results on the frame
    annotated_frame = results[0].plot()

    # Display the resulting frame
    cv2.imshow("Camera", annotated_frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) == ord("q"):
        break

# Release resources and close windows
cv2.destroyAllWindows()